Imports

In [2]:
!pip install transformers
!pip install datasets
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 18.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [3]:
import numpy as np
import pandas as pd
from sklearn import metrics 
import transformers
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction


Lectura de los csv como Datasets

In [4]:
data_files = {"train": "train.csv", "validation": "validation.csv"}
dataset = load_dataset("csv", data_files=data_files)

dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-74b482778c7e193e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'service', 'metric', 'objective', 'remedy', 'claim', 'exception', 'definition', 'obligation', 'right', 'neither'],
        num_rows: 117
    })
    validation: Dataset({
        features: ['text', 'service', 'metric', 'objective', 'remedy', 'claim', 'exception', 'definition', 'obligation', 'right', 'neither'],
        num_rows: 51
    })
})

In [5]:
dataset['train'].features

{'text': Value(dtype='string', id=None),
 'service': Value(dtype='int64', id=None),
 'metric': Value(dtype='int64', id=None),
 'objective': Value(dtype='int64', id=None),
 'remedy': Value(dtype='int64', id=None),
 'claim': Value(dtype='int64', id=None),
 'exception': Value(dtype='int64', id=None),
 'definition': Value(dtype='int64', id=None),
 'obligation': Value(dtype='int64', id=None),
 'right': Value(dtype='int64', id=None),
 'neither': Value(dtype='int64', id=None)}

In [6]:
example = dataset['train'][:10]
example

{'text': ['Last Updated: May 25, 2022 This Amazon Compute Service Level Agreement (this “SLA”) is a policy governing the use of Amazon Elastic Compute Cloud (“Amazon EC2”)* and applies separately to each account using Amazon EC2.',
  'In the event of a conflict between the terms of this SLA and the terms of the AWS Customer Agreement or other agreement with us governing your use of our Services (the “Agreement”), the terms and conditions of this SLA apply, but only to the extent of such conflict.',
  'Capitalized terms used herein but not defined herein shall have the meanings set forth in the Agreement.',
  '*For purposes of this SLA, Amazon EC2 includes any Amazon Elastic Graphics, Amazon Elastic Inference, and Elastic IP Address resources purchased with the relevant Amazon EC2 instance(s).',
  'SLAs AWS makes two SLA commitments for Amazon EC2: (1) a Region-Level SLA that governs Amazon EC2 deployed across multiple AZs or regions, and (2) an Instance-Level SLA that governs Amazon EC

In [7]:
labels = [label for label in dataset['train'].features.keys() if label not in ['text', 'obligation', 'right', 'neither']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['service',
 'metric',
 'objective',
 'remedy',
 'claim',
 'exception',
 'definition']

In [8]:
df = dataset['train'].to_pandas()
df.head()

,text,service,metric,objective,remedy,claim,exception,definition,obligation,right,neither
0,"Last Updated: May 25, 2022 This Amazon Compute...",1,1,0,0,0,0,0,0,0,1
1,In the event of a conflict between the terms o...,1,0,0,0,0,0,1,0,0,1
2,Capitalized terms used herein but not defined ...,1,0,0,0,0,0,0,0,0,1
3,"*For purposes of this SLA, Amazon EC2 includes...",1,1,0,0,0,0,0,0,0,1
4,SLAs AWS makes two SLA commitments for Amazon ...,1,1,0,0,0,0,0,0,0,1


In [9]:
# Contar el número de veces que cada etiqueta toma el valor 1
freq = df[labels].sum()

# Divide las frecuencias entre el número total de filas
freq = freq / len(df)
print(freq*100)

service       36.752137
metric        29.059829
objective     28.205128
remedy        10.256410
claim         28.205128
exception      8.547009
definition    22.222222
dtype: float64


Preprocess data

In [10]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [11]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

El tokenizer nos devuelve: 
  - **input_ids**: indica el orden de los inputs tokenizados que vamos a proporcionar al modelo
  - **token_type_ids**: forma de indicar al modelo de lenguaje natural el contexto de un token dentro de una frase o enunciado. Por ejemplo, en un modelo BERT, se utilizan dos tipos de tokens para indicar que una determinada secuencia de tokens proviene del primer segmento y otra para indicar que proviene del segundo.
  - **attention_mask**: es un vector que se usa en modelos de procesamiento de lenguaje natural como BERT para indicar a la red neuronal cuáles tokens en el input deben ser considerados para predecir la salida. Por ejemplo, si un token es un token de relleno agregado para que el input tenga un tamaño fijo, la máscara de atención marcará ese token como 0, lo que indica que no debe ser considerado por la red neuronal.

In [12]:
example = encoded_dataset['train'][0]
print(example)

{'input_ids': [101, 2197, 7172, 1024, 2089, 2423, 1010, 16798, 2475, 2023, 9733, 24134, 2326, 2504, 3820, 1006, 2023, 1523, 22889, 2050, 1524, 1007, 2003, 1037, 3343, 8677, 1996, 2224, 1997, 9733, 21274, 24134, 6112, 1006, 1523, 9733, 14925, 2475, 1524, 1007, 1008, 1998, 12033, 10329, 2000, 2169, 4070, 2478, 9733, 14925, 2475, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1,

In [13]:
tokenizer.decode(example['input_ids'])

'[CLS] last updated : may 25, 2022 this amazon compute service level agreement ( this “ sla ” ) is a policy governing the use of amazon elastic compute cloud ( “ amazon ec2 ” ) * and applies separately to each account using amazon ec2. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [19]:
example['labels']

[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [15]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['service', 'metric']

In [20]:
encoded_dataset.set_format("torch")

Definir modelo Bert

Usaremos el preentrenado con la configuración por defecto

In [21]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
!pip install huggingface_hub

In [31]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# repo_name = "bert-finetuned-sla"
# model.push_to_hub(repo_name)

In [ ]:
# tokenizer.push_to_hub(repo_name)

Entrenamos el modelo

Usaremos el método de entrenamiento de la API de Hugging face. Requiere dos cosas.

In [32]:
batch_size = 4
metric_name = "accuracy"

In [33]:
args = TrainingArguments(
    f"bert-finetuned-sla",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [35]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [36]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2197,  7172,  1024,  2089,  2423,  1010, 16798,  2475,  2023,
         9733, 24134,  2326,  2504,  3820,  1006,  2023,  1523, 22889,  2050,
         1524,  1007,  2003,  1037,  3343,  8677,  1996,  2224,  1997,  9733,
        21274, 24134,  6112,  1006,  1523,  9733, 14925,  2475,  1524,  1007,
         1008,  1998, 12033, 10329,  2000,  2169,  4070,  2478,  9733, 14925,
         2475,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [37]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

RuntimeError: ignored

In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/marmolpen3/bert-finetuned-sla into local empty directory.


Download file pytorch_model.bin:   0%|          | 1.44k/418M [00:00<?, ?B/s]

Download file runs/Feb06_00-30-08_de3dbc66d5af/events.out.tfevents.1675643796.de3dbc66d5af.276.2: 100%|#######…

Download file runs/Feb06_00-10-59_7ec881d9084b/1675642622.110147/events.out.tfevents.1675642622.7ec881d9084b.1…

Download file runs/Feb05_22-09-41_84475df473bd/events.out.tfevents.1675635002.84475df473bd.360.11: 100%|######…

...

Download file runs/Feb05_22-09-41_84475df473bd/1675635002.3081682/events.out.tfevents.1675635002.84475df473bd.…

Clean file runs/Feb05_22-09-41_84475df473bd/1675635002.3081682/events.out.tfevents.1675635002.84475df473bd.360…

Download file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##9       | 1.00k/3.43k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/418M [00:00<?, ?B/s]

In [39]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 117
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 600
  Number of trainable parameters = 109487623
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.499385,0.000000,0.500000,0.000000
2,No log,0.440801,0.000000,0.500000,0.000000
3,No log,0.376052,0.444444,0.646194,0.196078
4,No log,0.343754,0.649573,0.760381,0.470588
5,No log,0.327435,0.655462,0.766003,0.529412
6,No log,0.309307,0.655738,0.769896,0.470588
7,No log,0.308259,0.656000,0.773789,0.509804
8,No log,0.303015,0.645669,0.770329,0.470588
9,No log,0.309636,0.666667,0.781142,0.490196
10,No log,0.297646,0.671756,0.790657,0.509804


***** Running Evaluation *****
  Num examples = 51
  Batch size = 4
Saving model checkpoint to bert-finetuned-sla/checkpoint-30
Configuration saved in bert-finetuned-sla/checkpoint-30/config.json
Model weights saved in bert-finetuned-sla/checkpoint-30/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sla/checkpoint-30/tokenizer_config.json
Special tokens file saved in bert-finetuned-sla/checkpoint-30/special_tokens_map.json
tokenizer config file saved in bert-finetuned-sla/tokenizer_config.json
Special tokens file saved in bert-finetuned-sla/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 51
  Batch size = 4
Saving model checkpoint to bert-finetuned-sla/checkpoint-60
Configuration saved in bert-finetuned-sla/checkpoint-60/config.json
Model weights saved in bert-finetuned-sla/checkpoint-60/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sla/checkpoint-60/tokenizer_config.json
Special tokens file saved in bert-finetuned-sla/checkpoint

TrainOutput(global_step=600, training_loss=0.19871988773345947, metrics={'train_runtime': 259.9265, 'train_samples_per_second': 9.003, 'train_steps_per_second': 2.308, 'total_flos': 153926877312000.0, 'train_loss': 0.19871988773345947, 'epoch': 20.0})

Evaluate

After training, we evaluate our model on the validation set.

In [40]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 51
  Batch size = 4


{'eval_loss': 0.3274349272251129,
 'eval_f1': 0.6554621848739496,
 'eval_roc_auc': 0.7660034602076125,
 'eval_accuracy': 0.5294117647058824,
 'eval_runtime': 0.5137,
 'eval_samples_per_second': 99.274,
 'eval_steps_per_second': 25.305,
 'epoch': 20.0}

In [44]:
trainer.push_to_hub()

Saving model checkpoint to bert-finetuned-sla
Configuration saved in bert-finetuned-sla/config.json
Model weights saved in bert-finetuned-sla/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sla/tokenizer_config.json
Special tokens file saved in bert-finetuned-sla/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

Upload file runs/Feb06_02-26-44_3eb5eacccc76/events.out.tfevents.1675650615.3eb5eacccc76.265.0: 100%|#########…

Upload file runs/Feb06_02-26-44_3eb5eacccc76/events.out.tfevents.1675651097.3eb5eacccc76.265.2: 100%|#########…

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/marmolpen3/bert-finetuned-sla
   5865d6d..e3ecc64  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/marmolpen3/bert-finetuned-sla
   5865d6d..e3ecc64  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'F1', 'type': 'f1', 'value': 0.6554621848739496}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.5294117647058824}]}
To https://huggingface.co/marmolpen3/bert-finetuned-sla
   e3ecc64..ba8de65  main -> main

   e3ecc64..ba8de65  main -> main



'https://huggingface.co/marmolpen3/bert-finetuned-sla/commit/e3ecc649482371b37ed1700c92f9615e9c5cf8ac'

Inference

In [46]:
data_files = {"test": "test.csv"}
test_data = load_dataset("csv", data_files=data_files)

test_data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6b2c901eace8dd17/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 27
    })
})

In [48]:
sents_test = test_data['test'][:]["text"]

encoding = tokenizer(sents_test, return_tensors="pt", padding=True, truncation=True)
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [50]:
logits = outputs.logits
logits

tensor([[ 0.4174, -0.2711, -1.4114, -2.4995, -2.1799, -2.4467, -2.3151],
        [ 0.3356,  0.7225, -0.2761, -2.0065, -2.1244, -2.3188, -2.1837],
        [ 0.3928,  0.1928, -0.2237, -2.1189, -2.3269, -2.6316, -2.0399],
        [ 0.4874,  1.5220,  1.7934,  0.4790, -1.3562, -1.9346, -2.2651],
        [-1.2962, -1.1352, -1.9072, -2.2452,  1.2802, -1.2509, -1.8340],
        [ 0.6132, -1.4798, -1.0846, -2.1224, -0.8412, -2.2391, -2.7229],
        [-0.7629, -0.8799,  1.0073,  1.2543,  0.6428, -0.9678, -0.6556],
        [-0.7105, -0.8064,  0.9989,  1.3030,  0.5740, -0.8349, -0.5292],
        [-0.4833, -0.3227,  1.2678,  1.4381,  0.3319, -0.7514, -0.6618],
        [-1.3802, -1.7585, -2.3108, -2.2844,  1.1761, -0.9447, -1.6748],
        [-1.4536, -1.7317, -2.3051, -2.4043,  1.1587, -1.0328, -1.6303],
        [-1.2685, -2.6540, -3.0589, -2.7029, -0.3541, -1.0950,  0.4994],
        [-0.9022, -2.3639, -2.9859, -2.2616,  0.6614, -0.1056, -0.4797],
        [-0.9142, -2.0028, -2.3166, -2.3308,  0.912

In [52]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()

for logit in logits:
  probs = sigmoid(logit.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  print(probs)
  predictions[np.where(probs >= 0.5)] = 1
  # turn predicted id's into actual label names
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  print(predicted_labels)

tensor([0.6029, 0.4326, 0.1960, 0.0759, 0.1016, 0.0797, 0.0899],
       grad_fn=<SigmoidBackward0>)
['service']
tensor([0.5831, 0.6732, 0.4314, 0.1185, 0.1067, 0.0896, 0.1012],
       grad_fn=<SigmoidBackward0>)
['service', 'metric']
tensor([0.5970, 0.5481, 0.4443, 0.1073, 0.0889, 0.0671, 0.1151],
       grad_fn=<SigmoidBackward0>)
['service', 'metric']
tensor([0.6195, 0.8208, 0.8573, 0.6175, 0.2049, 0.1262, 0.0941],
       grad_fn=<SigmoidBackward0>)
['service', 'metric', 'objective', 'remedy']
tensor([0.2148, 0.2432, 0.1293, 0.0958, 0.7825, 0.2225, 0.1378],
       grad_fn=<SigmoidBackward0>)
['claim']
tensor([0.6487, 0.1855, 0.2526, 0.1069, 0.3013, 0.0963, 0.0616],
       grad_fn=<SigmoidBackward0>)
['service']
tensor([0.3180, 0.2932, 0.7325, 0.7780, 0.6554, 0.2753, 0.3417],
       grad_fn=<SigmoidBackward0>)
['objective', 'remedy', 'claim']
tensor([0.3295, 0.3087, 0.7308, 0.7863, 0.6397, 0.3026, 0.3707],
       grad_fn=<SigmoidBackward0>)
['objective', 'remedy', 'claim']
tensor([0.3